In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time

driver = webdriver.Chrome()
driver.get('https://tripsome.az/az/categories/all')
wait = WebDriverWait(driver, 8)
links = []
all_urls =[]
# Scroll down the page approximately 20 times
for _ in range(20):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(3)

for i in range(2, 128):
    xpath = f'//*[@id="app"]/div/main/div[2]/div/div[3]/div/div/div[{i}]/a'
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        link = element.get_attribute('href')
        links.append(link)
    except NoSuchElementException:
        print(f'Element not found: {xpath}')
all_urls= '\n'.join(links).split('\n')
print(all_urls)
driver.close()

In [ ]:
import pandas as pd
column_titles = [ 'Links is']
df = pd.DataFrame(all_urls, columns=column_titles)
print(df)

In [ ]:
import pandas as pd
column_titles = [ 'Links is']
df = pd.DataFrame(all_urls, columns=column_titles)
df.to_csv(r'C:\Users\mamma\Downloads\Tripsome_az_urls.csv', index=False)

In [127]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

In [128]:
data1=pd.read_csv("Tripsome_az_urls.csv")

In [129]:
data1

,Links is
0,https://tripsome.az/az/activities/f1b7455/futs...
1,https://tripsome.az/az/activities/f0c434d/cine...
2,https://tripsome.az/az/activities/9c50227/snay...
3,https://tripsome.az/az/activities/f180dfe/kart...
4,https://tripsome.az/az/activities/8c41954/mix-...
...,...
121,https://tripsome.az/az/activities/3c75f2b/elek...
122,https://tripsome.az/az/activities/7e1a6cf/kera...
123,https://tripsome.az/az/activities/5e14f98/tenn...
124,https://tripsome.az/az/activities/f6cba68/pesa...


In [130]:
r = requests.get(data1['Links is'][42])
soup = BeautifulSoup(r.content, 'html.parser')

In [131]:
data1=data1.drop(42,axis=0)

In [132]:
data1=data1.reset_index(drop=True)

In [133]:
data1

,Links is
0,https://tripsome.az/az/activities/f1b7455/futs...
1,https://tripsome.az/az/activities/f0c434d/cine...
2,https://tripsome.az/az/activities/9c50227/snay...
3,https://tripsome.az/az/activities/f180dfe/kart...
4,https://tripsome.az/az/activities/8c41954/mix-...
...,...
120,https://tripsome.az/az/activities/3c75f2b/elek...
121,https://tripsome.az/az/activities/7e1a6cf/kera...
122,https://tripsome.az/az/activities/5e14f98/tenn...
123,https://tripsome.az/az/activities/f6cba68/pesa...


In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
def activity_information(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    category=soup.find_all('div', attrs = {'class':'c-activity-page-category'})[0].text.replace("\n","").split(":")[1].strip()
    activity_name=soup.find_all('div', attrs = {'class':'c-activity-page-title'})[0].text
    price=soup.find_all('div', attrs = {'class':'c-activity-price'})[0].text.replace("\n","").split(" ")[1]+"AZN"
    age_range=soup.find_all('div', attrs = {'class':'c-activity-page-age text-secondary'})[0].text.replace(".\n","").strip().split("+")[0].strip()+"+"
    activity_description=soup.find_all('div', attrs = {'class':'c-activity-section-info'})[0].text
    selected = soup.find_all('activity-date-time')
    
    date = []
    for x in selected:
        date.append(str(x))

    def week_func(n_day):
        gun=''
        if n_day=='sa':
            gun="Şən."
        elif n_day=='su':
            gun="Baz"
        elif n_day=='m':
            gun="Baz.E."
        elif n_day=='tu':
            gun="Çərş.A."
        elif n_day=='w':
            gun="Çərş."
        elif n_day=='th':
            gun="Cüm.A."
        elif n_day=='f':
            gun="Cüm."

        return gun


    def month_func(n_month):
        ay=''
        if n_month=='04':
            ay="Apr."
        elif n_month=='05':
            ay="May"
        elif n_month=='06':
            ay="Iyn."
        elif n_month=='07':
            ay="Iyl."
        elif n_month=='08':
            ay="Avq."
        elif n_month=='09':
            ay="Sen."
        elif n_month=='10':
            ay="Okt."
        elif n_month=='11':
            ay="Noy." 
        elif n_month=='12':
            ay="Dek."
        return ay

    json_string = date[0]

    # Extract the JSON data from the string
    json_data = json.loads(json_string.split(":terms=\'")[1].split("\'>")[0])

    my_dict={}

    datelist=[]
    for i in json_data:
        lst=[]
        lst += [i['firstDay'][8:10]+" "+month_func(i['firstDay'][5:7])]
        lst += [i['lastDay'][8:10]+" "+month_func(i['lastDay'][5:7])]
        lst += [i['time_from']]
        lst += [i['time_to']]

        week=['m','tu','w','th','f','sa','su']


        for k in week:
            if i[k]==1:
                lst += [week_func(k)]

        datelist+= [lst]



    available_dates = []
    for i in datelist:
        lst = []
        lst += [i[4] + ' - ' + i[-1] + ', ' + i[2][:5]+ '-' + i[3][:5]+' ('+ i[0][0:]+ ' - '+ i[1][0:] + ')' ]
        available_dates += lst 
    activity_information_dict={"Category":category,"Activity_name":activity_name,"Price":price,"Age_range":age_range,"Activity_description":activity_description,"Available_dates":available_dates}
    return activity_information_dict

In [36]:
from tqdm import tqdm

In [37]:
list_of_activity_information=[]
for i in tqdm(range(len(data1['Links is']))):
    list_of_activity_information.append(activity_information(data1['Links is'][i]))
        

100%|████████████████████████████████████████████████████████████████████████████████| 125/125 [01:18<00:00,  1.59it/s]


In [52]:
import json

# Open a file for writing
with open('list_of_activity_information.json', 'w') as f:
    # Write the list to the file using the json.dump method
    json.dump(list_of_activity_information, f)

In [8]:
with open('list_of_activity_information.json', 'r') as f:
    # Load the contents of the file using the json.load method
    my_list = json.load(f)

In [9]:
len(my_list)

125

In [8]:
import csv
keys = list_of_activity_information[0].keys()
filename = open("list_of_activity_information.csv", "w",encoding="utf-8")
a = csv.DictWriter(filename,keys)
a.writeheader()
a.writerows(list_of_activity_information)
filename.close()
import pandas as pd
data = pd.read_csv('list_of_activity_information.csv')
data

NameError: name 'list_of_activity_information' is not defined

In [134]:
data = pd.read_csv('list_of_activity_information.csv')
data

,Category,Activity_name,Price,Age_range,Activity_description,Available_dates
0,İdman və Döyüş Sənəti,Futsal Meetup (hər həftə),7AZN,14+,Biletə daxildir:\r\n\r\nOyun üçün xalça sahə\r...,"['Cüm.A. - Cüm.A., 22:00-23:00 (27 Apr. - 28 D..."
1,"Tamaşa, Teatr, Şoular","Cinema Fulldome - ""Kəşf Edilməyən Dünyalar""",3AZN,3+,\r\nMüddət: 30 dəqiqə\r\nTarix: Hər şənbə və b...,"['Şən. - Baz, 17:00-17:30 (22 Apr. - 25 Iyn.)']"
2,İdman və Döyüş Sənəti,"Snayper - Nəfəs, Dəqiqlik və Atəş",14AZN,7+,Qiymətə daxildir\r\n\r\nSilahla düzgün davranı...,"['Çərş.A. - Cüm., 12:00-20:00 (25 Apr. - 29 De..."
3,İdman və Döyüş Sənəti,Kartinq Sür və Formula 1 həyəcanını hiss et,15AZN,10+,Qiymətə daxildir:\r\n\r\nTəhlükəsizlik brifinq...,"['Baz.E. - Baz, 12:00-00:00 (22 Apr. - 31 Dek.)']"
4,İdman və Döyüş Sənəti,Mix - Snayper və Avtomat Airsoft,18AZN,7+,Qiymətə daxildir\r\n\r\nSilahla düzgün davranı...,"['Çərş.A. - Baz, 12:00-20:00 (22 Apr. - 31 Dek..."
...,...,...,...,...,...,...
120,Turlar və Səyahət,Elektrik Velosipedi Turu,100AZN,14+,Tur elektrik velosipedləri ilə həyata keçirili...,"['Baz - Baz, 07:00-18:00 (23 Apr. - 31 Dek.)']"
121,İncəsənət və Sənətkarlıq,Keramika -Gil Yapma Sənəti,30AZN,5+,Studioda keramika və rəngkarlıq dərsləri keçir...,"['Baz.E. - Baz, 10:00-19:00 (22 Apr. - 31 Dek...."
122,İdman və Döyüş Sənəti,Tennis Masterklassı,10AZN,5+,Sizi tennis üzrə Azərbaycan Çempionatının qali...,"['Çərş. - Çərş., 09:00-11:00 (26 Apr. - 27 Dek..."
123,İdman və Döyüş Sənəti,Peşakar Atçılıq Kursu,40AZN,7+,Peşakar Atçılıq Təlimləri\r\nTəsvir\r\nAt sürm...,"['Baz.E. - Baz, 10:00-19:00 (22 Apr. - 31 Dek.)']"


In [135]:
data["Price"]=data["Price"].str.split("AZN",expand=True)[0]
data["Price"]=pd.to_numeric(data["Price"])

In [136]:
data["Links"]=data1["Links is"]

In [34]:
data

,Category,Activity_name,Price,Age_range,Activity_description,Available_dates,Links
0,İdman və Döyüş Sənəti,Futsal Meetup (hər həftə),7,14+,Biletə daxildir:\r\n\r\nOyun üçün xalça sahə\r...,"['Cüm.A. - Cüm.A., 22:00-23:00 (27 Apr. - 28 D...",https://tripsome.az/az/activities/f1b7455/futs...
1,"Tamaşa, Teatr, Şoular","Cinema Fulldome - ""Kəşf Edilməyən Dünyalar""",3,3+,\r\nMüddət: 30 dəqiqə\r\nTarix: Hər şənbə və b...,"['Şən. - Baz, 17:00-17:30 (22 Apr. - 25 Iyn.)']",https://tripsome.az/az/activities/f0c434d/cine...
2,İdman və Döyüş Sənəti,"Snayper - Nəfəs, Dəqiqlik və Atəş",14,7+,Qiymətə daxildir\r\n\r\nSilahla düzgün davranı...,"['Çərş.A. - Cüm., 12:00-20:00 (25 Apr. - 29 De...",https://tripsome.az/az/activities/9c50227/snay...
3,İdman və Döyüş Sənəti,Kartinq Sür və Formula 1 həyəcanını hiss et,15,10+,Qiymətə daxildir:\r\n\r\nTəhlükəsizlik brifinq...,"['Baz.E. - Baz, 12:00-00:00 (22 Apr. - 31 Dek.)']",https://tripsome.az/az/activities/f180dfe/kart...
4,İdman və Döyüş Sənəti,Mix - Snayper və Avtomat Airsoft,18,7+,Qiymətə daxildir\r\n\r\nSilahla düzgün davranı...,"['Çərş.A. - Baz, 12:00-20:00 (22 Apr. - 31 Dek...",https://tripsome.az/az/activities/8c41954/mix-...
...,...,...,...,...,...,...,...
120,Turlar və Səyahət,Elektrik Velosipedi Turu,100,14+,Tur elektrik velosipedləri ilə həyata keçirili...,"['Baz - Baz, 07:00-18:00 (23 Apr. - 31 Dek.)']",https://tripsome.az/az/activities/3c75f2b/elek...
121,İncəsənət və Sənətkarlıq,Keramika -Gil Yapma Sənəti,30,5+,Studioda keramika və rəngkarlıq dərsləri keçir...,"['Baz.E. - Baz, 10:00-19:00 (22 Apr. - 31 Dek....",https://tripsome.az/az/activities/7e1a6cf/kera...
122,İdman və Döyüş Sənəti,Tennis Masterklassı,10,5+,Sizi tennis üzrə Azərbaycan Çempionatının qali...,"['Çərş. - Çərş., 09:00-11:00 (26 Apr. - 27 Dek...",https://tripsome.az/az/activities/5e14f98/tenn...
123,İdman və Döyüş Sənəti,Peşakar Atçılıq Kursu,40,7+,Peşakar Atçılıq Təlimləri\r\nTəsvir\r\nAt sürm...,"['Baz.E. - Baz, 10:00-19:00 (22 Apr. - 31 Dek.)']",https://tripsome.az/az/activities/f6cba68/pesa...


In [151]:
import re
def find_available_activity():
    category=input("zehmet olmasa kategory secin:")
    price=input("zehmet olmasa qiymet secin:")
    price_list=price.split("-")

    special_request=input("xususi istekleriniz varmi?:")
    special_request_list=special_request.split(",")
    
    k=0
    mylist=[]

    for i in range(len(data)):
        if len(special_request_list)==1 and special_request_list[0]==' ':
            if category==data['Category'][i] and int(price_list[0])<=data["Price"][i]<=int(price_list[1]):
                k+=1
                mylist += [i]
        else:
            if category==data['Category'][i] and int(price_list[0])<=data["Price"][i]<=int(price_list[1]):
                list_ = []
                for j in special_request_list:
                    if j=="samovar":
                        matches = re.findall(r'\b\w*sam\w*var\w*\b', (data['Activity_description'][i]).lower())
                        list_ += matches
                    else:
                        matches = re.findall(f"{j.lower()}", (data['Activity_description'][i]).lower())
                        list_ += matches

                    if len(list_)>0:
                        k+=1
                        mylist += [[i] + list(set(list_))]
        
    if k==0:
        print("isteyinize uygun aktivlik tapilmadi")
    
    else:
        if len(special_request_list)==1 and special_request_list[0]==' ':
            print(f"\nIstəyinize uygun {len(mylist)} aktivlik tapildi.\n")
            for k in mylist:
                print("Activity_name:",data["Activity_name"][k],"\n","Price:",str(data["Price"][k])+"AZN","\n","Age_range:",data['Age_range'][k],"\n",'Activity_description:',data['Activity_description'][k],"\n","Available_dates:",data['Available_dates'][k], '\n',"access for activity:",data["Links"][k],"\n")
      
        else:
            max_len_index = max(range(len(mylist)), key=lambda i: len(mylist[i]))
            percentage = (len(mylist[max_len_index])-1)/(len(special_request_list))
            count = sum((1 for k in mylist if len(k)==len(mylist[max_len_index])))
            if percentage!=1:
                print(f"\nIstəyinize uygun  {round(percentage*100,2)} % deqiqlikle {count} aktivlik tapildi.\n")
            else:
                print(f"\nIstəyinize uygun {count} aktivlik tapildi.\n")
            for k in mylist:
                if len(k)==len(mylist[max_len_index]):
                    print("Activity_name:",data["Activity_name"][k[0]],"\n","Price:",str(data["Price"][k[0]])+"AZN","\n","Age_range:",data['Age_range'][k[0]],"\n",'Activity_description:',data['Activity_description'][k[0]],"\n","Available_dates:",data['Available_dates'][k[0]], '\n',"access for activity:",data["Links"][k[0]],"\n")

In [155]:
find_available_activity()

zehmet olmasa kategory secin:Turlar və Səyahət
zehmet olmasa qiymet secin:30-50
xususi istekleriniz varmi?:əyləncə,kitab,adrenalin,hədiyyə

Istəyinize uygun  50.0 % deqiqlikle 5 aktivlik tapildi.

Activity_name: Ezraider Turu - Yanardağ 
 Price: 40AZN 
 Age_range: 18+ 
 Activity_description: Biletə daxildir:

Təhlükəsizlik brifinqi
Qoruyucu kaska (baş geyimi)
Ezraider skuteri (45 dəqiqə)
5 ox atışı
5 nizə atışı
Tur bələdçisi
Xüsusi dizayn olunmuş tur marşrutun (uzunluğu 6.5km)

Qiymət: 40AZN (1 nəfər; 45 dəqiqəlik ezraider + 5 ox atışı + 5 nizə atışı)
Qeyd: Qrup kimi eyni vaxtda maksimum 10 nəfər iştirak edə bilər
Ünvan: Yanardağ qoruğu
Ləğv qaydası: Xidmətin baş tutmasına 96 saat və ya daha çox müddət qaldıqda bilet geri qaytarılarsa və ya dəyişdirilərsə, bilet dəyərinin 20%-i tutulur. Xidmətin baş tutmasına 96 saat və ya daha az müddət qaldıqda bilet geri qaytarılmır və ya dəyişdirilmir.
Ezraider Haqqında
Sizi daha əvvəl yaşamadığınız hissləri təcrübə etməyə çağırırıq. Kompakt və uni